In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, timezone
import requests
import os

In [2]:
path_dir = "/home/alberto/Docker/Trading/analysis/json"
list_json = os.listdir(path_dir)
full_paths = [path_dir + "/{0}".format(x) for x in list_json]

list_json_info = []
for full_path in full_paths:
    json_ = {'path': full_path, 'time': os.path.getmtime(full_path)}
    list_json_info.append(json_)

list_json_info.sort(key=lambda x: x['time'], reverse=False)
list_json_info

data_dict = {}
for json_info in list_json_info:
    path = json_info['path']
    print(f'Retrieving data from {path}')
    f = open(path, "r")
    if len(data_dict) == 0:
        data_dict = json.loads(f.read())
        #print(data_dict['data']['BTCUSDT'][-1]['_id'])
    else:
        temp_data_dict = json.loads(f.read())
        #print(f'{temp_data_dict['data']['BTCUSDT'][0]['_id']})
        for coin in temp_data_dict['data']:
            if coin not in data_dict['data']:
                data_dict['data'][coin] = []
            for obs in temp_data_dict['data'][coin]:
                data_dict['data'][coin].append(obs)
data = data_dict['data']

Retrieving data from /home/alberto/Docker/Trading/analysis/json/data-03-04-2023.json
2023-04-10T23:48:05.511417
Retrieving data from /home/alberto/Docker/Trading/analysis/json/data-12-4-2023.json
2023-04-12T13:06:05.383969


In [3]:
print(temp_data_dict['data']['BTCUSDT'][0]['_id'])

2023-04-10T23:49:05.727023


In [ ]:
# FETCH TRACKER DATA
path = "/home/alberto/Docker/Trading/analysis/json/data.json"
f = open (path, "r")
data_dict = json.loads(f.read())
data = data_dict['data']
print('DATETIME CREATION: ', data_dict['datetime_creation'])

# FETCH MOST TRADED COINS
path = "/home/alberto/Docker/Trading/tracker/json/most_traded_coins.json"
f = open (path, "r")
most_traded_coins = json.loads(f.read())
most_traded_coins = most_traded_coins['most_traded_coins']

In [ ]:
#['_id','price', price_%_1d','price_%_6h','price_%_3h','price_%_1h',
#'vol_1m', 'buy_vol_1m', buy_trd_1m
#'vol_5m','vol_5m_std','buy_vol_5m','buy_vol_5m_std','buy_trd_5m','buy_trd_5m_std',
#'vol_15m','vol_15m_std','buy_vol_15m','buy_vol_15m_std','buy_trd_15m','buy_trd_15m_std',
#'vol_30m','vol_30m_std','buy_vol_30m','buy_vol_30m_std','buy_trd_30m','buy_trd_30m_std',
#'vol_60m','vol_60m_std','buy_vol_60m','buy_vol_60m_std','buy_trd_60m','buy_trd_60m_std',
#'vol_24h','vol_24h_std','buy_vol_24h','buy_vol_24h_std','buy_trd_24h','buy_trd_24h_std']

In [ ]:
def search_focus_points(data, most_traded_coins,  days_start=4, days_finish=3, volume_limit=0.8, buy_volume_limit=0.55, referenceTime_volume='vol_5m',
                         referenceTime_Buyvolume='buy_vol_5m', filter_position=50):
    '''
    This function looks for all those points in time for each coin that have interesting events.
    '''

    most_traded_coins = most_traded_coins[:filter_position]

    minutes_start = int(days_start*24*60)
    if days_finish:
        minutes_end = int(days_finish*24*60)
    else:
        minutes_end = None

    events = {}
    for coin in data:
        if coin in most_traded_coins:
            i = 0
            for obs in data[coin][-minutes_start:minutes_end]:
                try:
                    i += 1
                    if obs[referenceTime_volume] and obs[referenceTime_volume] >= volume_limit and obs[referenceTime_Buyvolume] >= buy_volume_limit:
                        if coin not in events:
                            events[coin] = []
                        obs['position'] = most_traded_coins.index(coin)
                        obs['timeseries_position'] = i
                        events[coin].append(obs)
                except:
                    pass


    
    n_coins = len(events)
    print(f'Total Coins: {n_coins}')
    for coin in events:
        len_events_coin = len(events[coin])
        position = events[coin][0]['position']
        datetime_first_event = events[coin][0]['_id']
        print(f'{len_events_coin} events for {coin} in position {position}')
        print(f'First event at {datetime_first_event}')


    # minutes past from which evaluate price changes
    reference_time = [5, 10, 15, 20, 25, 30, 40, 50, 60, 90, 120, 240, 480, 720, 1440, 2880, 5760, 10080]
    reference_time = [5, 10, 15, 20, 25, 30, 40, 50, 60, 90, 120, 240, 480, 720, 1440, 2880, 5760]
    reference_time = [5, 10, 15, 20, 25, 30, 40, 50, 60, 90, 120, 240, 480, 720, 1440]
    #reference_time = [5]

    results = {}
    for coin in events:
        for event in events[coin]:
            initial_price = event['price']
            iso_timestamp_start = event['_id']
            datetime_initial_price = datetime.fromisoformat(iso_timestamp_start)
            for minutes in reference_time:
                estimated_iso_timestamp_end = (datetime_initial_price + timedelta(minutes=minutes)).isoformat()
                position_timeseries = event['timeseries_position']
                timeseries = data[coin][-minutes_start:minutes_end]
                final_price = timeseries[position_timeseries]['price']
                real_iso_timestamp_end = timeseries[position_timeseries+minutes]['_id']
                print(f'Estimated ISO timestamp: {estimated_iso_timestamp_end}')
                print(f'Real ISO timestamp: {real_iso_timestamp_end}')
    
                
    # TODO: CHECK IF THE COIN IS CONSISTENT IN BENCHMARK
        

        
    return events


events = search_focus_points(data, most_traded_coins,  days_start=2, days_finish=1, volume_limit=6, buy_volume_limit=0.55, referenceTime_volume='vol_5m',
                         referenceTime_Buyvolume='buy_vol_5m', filter_position=50)



In [ ]:
events['IOSTUSDT']